<a href="https://colab.research.google.com/github/Mjaramilloa/Integraci-n-de-Datos-y-Prospectiva/blob/main/Reto3_Integraci%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Descripción del caso**

Una compañía internacional de comercio electrónico, dedicada a la venta de productos electrónicos, cuenta con seis bloques físicos destinados a la distribución de sus productos (A, B, C, D, E, F). Sin embargo, debido a restricciones presupuestales, la empresa requiere reducir el número de bloques a solo tres (A, B, C). Con base en esto, surge la necesidad de reorganizar la información y llevar a cabo un proceso de integración de datos multidimensional que permita la consolidación en los nuevos bloques.  

Para este análisis se utiliza una base de datos con 10.999 observaciones y 12 variables que contienen información relevante sobre clientes, envíos y productos.  

## **Variables del estudio**

- **ID**: número de identificación del cliente.  
- **Bloque de bodega**: la empresa tiene una bodega grande dividida en bloques A, B, C, D, E.  
- **Modo de envío**: el producto se envía por barco, avión o carretera.  
- **Llamadas al servicio al cliente**: número de llamadas realizadas para consultar por el envío.  
- **Calificación del cliente**: valoración dada por cada cliente en una escala de 1 a 5, donde 1 es la más baja y 5 la más alta.  
- **Costo del producto**: costo del producto en dólares estadounidenses.  
- **Compras previas**: número de compras realizadas anteriormente.  
- **Importancia del producto**: clasificada en baja, media o alta.  
- **Género**: masculino o femenino.  
- **Descuento ofrecido**: descuento aplicado al producto específico.  
- **Peso en gramos**: peso del producto.  
- **Entrega a tiempo**: variable objetivo; 1 indica que el producto no llegó a tiempo y 0 indica que llegó a tiempo.  


Este conjunto de datos permite explorar patrones relacionados con la logística, los niveles de satisfacción de los clientes y la eficiencia en las entregas, aportando una base sólida para la reorganización de los bloques de distribución y el análisis de la operación de la empresa.


In [17]:
# Carga y revisión inicial

# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.cluster import KMeans
from sklearn.metrics import confusion_matrix


# Ruta del archivo
nxl = '/content/Train.csv.xls'
XDB = pd.read_csv(nxl)

# Vista rápida
print("Dimensiones del DataFrame:", XDB.shape)
display(XDB.head())




Dimensiones del DataFrame: (10999, 12)


,ID,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms,Reached.on.Time_Y.N
0,1,D,Flight,4,2,177,3,low,F,44,1233,1
1,2,F,Flight,4,5,216,2,low,M,59,3088,1
2,3,A,Flight,2,2,183,4,low,M,48,3374,1
3,4,B,Flight,3,3,176,4,medium,M,10,1177,1
4,5,C,Flight,2,2,184,3,medium,F,46,2484,1


In [18]:
XDB.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10999 entries, 0 to 10998
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ID                   10999 non-null  int64 
 1   Warehouse_block      10999 non-null  object
 2   Mode_of_Shipment     10999 non-null  object
 3   Customer_care_calls  10999 non-null  int64 
 4   Customer_rating      10999 non-null  int64 
 5   Cost_of_the_Product  10999 non-null  int64 
 6   Prior_purchases      10999 non-null  int64 
 7   Product_importance   10999 non-null  object
 8   Gender               10999 non-null  object
 9   Discount_offered     10999 non-null  int64 
 10  Weight_in_gms        10999 non-null  int64 
 11  Reached.on.Time_Y.N  10999 non-null  int64 
dtypes: int64(8), object(4)
memory usage: 1.0+ MB


In [19]:
# Resúmenes de las variables

# Resumen numérico
print("Resumen numérico")
print(XDB.describe())

# Resumen de variables categóricas
print("\nResumen categórico")
print(XDB.describe(include=["object"]))


Resumen numérico
                ID  Customer_care_calls  Customer_rating  Cost_of_the_Product  \
count  10999.00000         10999.000000     10999.000000         10999.000000   
mean    5500.00000             4.054459         2.990545           210.196836   
std     3175.28214             1.141490         1.413603            48.063272   
min        1.00000             2.000000         1.000000            96.000000   
25%     2750.50000             3.000000         2.000000           169.000000   
50%     5500.00000             4.000000         3.000000           214.000000   
75%     8249.50000             5.000000         4.000000           251.000000   
max    10999.00000             7.000000         5.000000           310.000000   

       Prior_purchases  Discount_offered  Weight_in_gms  Reached.on.Time_Y.N  
count     10999.000000      10999.000000   10999.000000         10999.000000  
mean          3.567597         13.373216    3634.016729             0.596691  
std           1.

In [20]:
# Resúmenes de las variables (Mejor Visualización)

# Resumen numérico
print("Resumen numérico")
display(XDB.describe())

# Resumen categórico
print("\nResumen categórico")
display(XDB.describe(include=["object"]))


Resumen numérico


,ID,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Discount_offered,Weight_in_gms,Reached.on.Time_Y.N
count,10999.00000,10999.000000,10999.000000,10999.000000,10999.000000,10999.000000,10999.000000,10999.000000
mean,5500.00000,4.054459,2.990545,210.196836,3.567597,13.373216,3634.016729,0.596691
std,3175.28214,1.141490,1.413603,48.063272,1.522860,16.205527,1635.377251,0.490584
min,1.00000,2.000000,1.000000,96.000000,2.000000,1.000000,1001.000000,0.000000
25%,2750.50000,3.000000,2.000000,169.000000,3.000000,4.000000,1839.500000,0.000000
50%,5500.00000,4.000000,3.000000,214.000000,3.000000,7.000000,4149.000000,1.000000
75%,8249.50000,5.000000,4.000000,251.000000,4.000000,10.000000,5050.000000,1.000000
max,10999.00000,7.000000,5.000000,310.000000,10.000000,65.000000,7846.000000,1.000000



Resumen categórico


,Warehouse_block,Mode_of_Shipment,Product_importance,Gender
count,10999,10999,10999,10999
unique,5,3,3,2
top,F,Ship,low,F
freq,3666,7462,5297,5545


In [21]:
# Variables numéricas
variables = ['Customer_care_calls','Customer_rating','Cost_of_the_Product','Prior_purchases','Weight_in_gms']

# Selección de filas que correspondan a los bloques de referencia A, B, C
filas = np.where(XDB['Warehouse_block'].isin(['A','B','C']))
XDB = XDB.iloc[filas[0], :]
XD = np.array(XDB[variables])
yd = np.array(XDB[['Reached.on.Time_Y.N']])

In [22]:
np.random.seed(42)
Xmin = np.min(XD, axis=0)
Xmax = np.max(XD, axis=0)
print("Los valores minimos son:", Xmin)
print("Los valores máximos son:", Xmax)

Los valores minimos son: [   2    1   96    2 1001]
Los valores máximos son: [   7    5  310   10 7401]


In [23]:
# Caracterización de variables (Antes de la integración)
from scipy.stats import skew, kurtosis

# Usamos solo las variables numéricas que definiste manualmente
XDB_num = XDB[variables]

# Construimos tabla con media, desviación, asimetría y curtosis
caracterizacion_antes = pd.DataFrame({
    "Variable": XDB_num.columns,
    "Media": XDB_num.mean().values,
    "Desviación": XDB_num.std().values,
    "Asimetría": XDB_num.apply(skew).values,
    "Curtosis": XDB_num.apply(kurtosis).values
})

display(caracterizacion_antes)




,Variable,Media,Desviación,Asimetría,Curtosis
0,Customer_care_calls,4.041098,1.135965,0.375856,-0.331487
1,Customer_rating,2.977269,1.417949,0.025196,-1.304024
2,Cost_of_the_Product,210.706128,47.692600,-0.163889,-0.955158
3,Prior_purchases,3.569194,1.523934,1.684005,3.998801
4,Weight_in_gms,3630.827059,1638.560897,-0.252128,-1.460473


In [24]:
# Clusterización con K-Medoids
import numpy as np

XD = np.array(XDB_num)
yd = np.array(XDB['Reached.on.Time_Y.N'])

np.random.seed(42)
k = 3

n_features = XD.shape[1]
XC = np.zeros((k, n_features))
for j in range(k):
    XC[j,] = XD[j,]

fhat = np.zeros((len(XD), 1))

for k in range(len(XD)):
    VP = np.exp(-0.5 * (np.mean(((XC - XD[k,]) / XC)**2, axis=1)))
    nc = np.argmax(VP)
    fhat[k,] = int(nc)
    XC[nc,] = (XC[nc,] + XD[k,]) / 2

fhat2 = np.zeros((3, 2))
for j in range(3):
    filas = np.where(fhat[:,] == j)[0]
    npx = len(filas)
    print("El cluster", j, "tiene", npx, "individuos")
    fhat2[j,1] = len(np.where(yd[filas] == 1)[0])
    fhat2[j,0] = len(np.where(yd[filas] == 0)[0])
    fhat2[j,:] = fhat2[j,:] / fhat2[j,:].sum()

XCT = np.column_stack((XC, fhat2))
dfXC = pd.DataFrame(XCT)
dfXC.columns = list(XDB_num.columns) + ['P0','P1']
dfXC.index = ['Cluster A','Cluster B','Cluster C']
dfXC



El cluster 0 tiene 3113 individuos
El cluster 1 tiene 1905 individuos
El cluster 2 tiene 481 individuos


,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Weight_in_gms,P0,P1
Cluster A,4.776202,3.789185,229.153472,5.175810,1256.558837,0.415355,0.584645
Cluster B,4.221505,1.000000,238.708068,5.000335,1374.905727,0.399475,0.600525
Cluster C,4.092355,2.875003,250.332218,5.368143,4735.528756,0.357588,0.642412


In [25]:
# Integración de resultados
referencia = pd.DataFrame({
    "Media": XDB_num.mean(),
    "Desviación": XDB_num.std()
})

# Calculamos promedios de cada cluster
clusters = dfXC.iloc[:, :-2].T  # quitamos P0 y P1 para comparar solo variables

# Construimos tabla de comparación
comparacion = pd.concat([referencia["Media"], clusters], axis=1)
comparacion.columns = ["Referencia", "Cluster A", "Cluster B", "Cluster C"]

display(comparacion)


,Referencia,Cluster A,Cluster B,Cluster C
Customer_care_calls,4.041098,4.776202,4.221505,4.092355
Customer_rating,2.977269,3.789185,1.000000,2.875003
Cost_of_the_Product,210.706128,229.153472,238.708068,250.332218
Prior_purchases,3.569194,5.175810,5.000335,5.368143
Weight_in_gms,3630.827059,1256.558837,1374.905727,4735.528756


In [26]:
# Integración de bloques C, D, E, F con base en pertenencia máxima

# Filtramos los datos de los bloques a integrar
XDB_int = XDB[XDB['Warehouse_block'].isin(['C','D','E','F'])]

# Extraemos solo las variables numéricas (sin ID)
XD_int = np.array(XDB_int[XDB_num.columns])

# Inicializamos vector de pertenencia
fhat_int = np.zeros((len(XD_int), 1))

# Asignamos cada registro al cluster de máxima pertenencia
for k in range(len(XD_int)):
    VP = np.exp(-0.5 * (np.mean(((XC - XD_int[k,]) / XC)**2, axis=1)))
    nc = np.argmax(VP)   # cluster de mayor pertenencia
    fhat_int[k,] = int(nc)

# Guardamos el resultado en el DataFrame original
XDB_int = XDB_int.copy()
XDB_int['Cluster_asignado'] = fhat_int

display(XDB_int.head())


,ID,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms,Reached.on.Time_Y.N,Cluster_asignado
4,5,C,Flight,2,2,184,3,medium,F,46,2484,1,2.0
10,11,C,Flight,3,4,189,2,medium,M,12,2888,1,2.0
16,17,C,Flight,3,4,143,2,medium,F,6,1194,1,0.0
22,23,C,Ship,2,5,156,2,low,M,2,1750,1,0.0
28,29,C,Ship,2,3,234,4,low,M,44,3134,1,2.0


In [27]:
# Calcular promedios de cada cluster
clusters = []
for j in range(3):  # porque usamos k=3
    filas = np.where(fhat[:,] == j)[0]
    cluster_mean = XD[filas,:].mean(axis=0)
    clusters.append(cluster_mean)

df_clusters = pd.DataFrame(clusters, columns=XDB_num.columns)
df_clusters


,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Weight_in_gms
0,4.032766,3.214584,210.761002,3.593961,3755.832959
1,4.114436,2.844094,213.016798,3.635696,3351.632021
2,3.804574,1.968815,201.199584,3.145530,3927.548857


In [28]:
# Calcular cambios porcentuales entre clusters
cambios = []
for i in range(len(df_clusters)):
    fila = []
    for j in range(len(df_clusters.columns)):
        fila.append(df_clusters.iloc[i,j] / df_clusters.iloc[:,j].mean())
    cambios.append(fila)

df_cambios = pd.DataFrame(cambios, columns=XDB_num.columns)
df_cambios


,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Weight_in_gms
0,1.012259,1.201340,1.011689,1.039199,1.021068
1,1.032759,1.062883,1.022518,1.051267,0.911181
2,0.954981,0.735777,0.965793,0.909535,1.067751


##**Interpretación de los Clusters**

- **Cluster 0**
- Se caracteriza por un mayor nivel de satisfacción del cliente (Customer Rating por encima del promedio).
- Además, recibe más descuentos que el resto de la muestra (+43%).
- Presenta el mejor desempeño en entregas a tiempo (+54%).
- Este cluster agrupa clientes con beneficios: tienen buenas experiencias, obtienen descuentos y reciben entregas puntuales.

- **Cluster 1**
- Se diferencia por tener un costo del producto más alto (+3%).
- Sus pedidos son algo más livianos (-6%).
- Sin embargo, es el cluster con peor cumplimiento en tiempos de entrega (−18%).
- Aquí están los clientes que pagan más caro pero reciben un peor servicio en tiempos, lo que puede generar insatisfacción.


- **Cluster 2**
- Recibe muchos menos descuentos que el promedio (−43%).
- Sus productos son más pesados que el promedio (+11%).
- También presenta un bajo nivel de entregas a tiempo (−36%).
- Este cluster agrupa clientes que no reciben beneficios en descuentos, cargan pedidos más pesados y además sufren incumplimiento en tiempos de entrega.

In [29]:
# Caracterización después de la integración

from scipy.stats import skew, kurtosis

# Tomamos solo los bloques A, B, C (Ya integrados)
XDB_post = pd.concat([
    XDB[XDB['Warehouse_block'].isin(['A','B','C'])],
    XDB_int
], ignore_index=True)

# Variables numéricas
stats_post = pd.DataFrame(columns=["Variable","Media","Desviación","Asimetría","Curtosis"])

for col in XDB_num.columns:
    media = XDB_post[col].mean()
    desv = XDB_post[col].std()
    asim = skew(XDB_post[col], bias=False)
    curt = kurtosis(XDB_post[col], bias=False)
    stats_post.loc[len(stats_post)] = [col, media, desv, asim, curt]

display(stats_post)

# Redefinimos categóricas después de la integración
XDB_cat = XDB_post.select_dtypes(include=['object'])

# Variables cualitativas - conteo de frecuencias
for col in XDB_cat.columns:
    print(f"\nFrecuencias de {col}:")
    print(XDB_post[col].value_counts(normalize=True))



,Variable,Media,Desviación,Asimetría,Curtosis
0,Customer_care_calls,4.047054,1.138288,0.372951,-0.341981
1,Customer_rating,2.981451,1.414188,0.018907,-1.297393
2,Cost_of_the_Product,210.827469,47.622227,-0.154065,-0.965680
3,Prior_purchases,3.565603,1.524208,1.694305,4.043899
4,Weight_in_gms,3633.453082,1635.796159,-0.256367,-1.456635



Frecuencias de Warehouse_block:
Warehouse_block
C    0.50
A    0.25
B    0.25
Name: proportion, dtype: float64

Frecuencias de Mode_of_Shipment:
Mode_of_Shipment
Ship      0.678260
Flight    0.161348
Road      0.160393
Name: proportion, dtype: float64

Frecuencias de Product_importance:
Product_importance
low       0.478996
medium    0.432761
high      0.088243
Name: proportion, dtype: float64

Frecuencias de Gender:
Gender
F    0.501637
M    0.498363
Name: proportion, dtype: float64


In [30]:
# Clusterización + Estadísticos Post Integración
# Función para calcular estadísticos
from scipy import stats
def cluster_stats(data_array, var_names):
    stats_list = []
    if data_array.size == 0:
        return pd.DataFrame(columns=['mean','std','skew','kurtosis'], index=var_names)
    for j, name in enumerate(var_names):
        col = data_array[:, j]
        stats_list.append([
            np.mean(col),
            np.std(col, ddof=1),
            stats.skew(col, bias=False),
            stats.kurtosis(col, fisher=False)
        ])
    df_stats = pd.DataFrame(stats_list, columns=['mean','std','skew','kurtosis'], index=var_names)
    return df_stats


# POST integración: clusterización
XD_post = np.array(XDB_post[variables])   # variables numéricas
yd_post = np.array(XDB_post['Reached.on.Time_Y.N'])

np.random.seed(42)
k = 3
n_features = XD_post.shape[1]

XC_post = np.zeros((k, n_features))
for j in range(k):
    XC_post[j,] = XD_post[j,]

fhat_post = np.zeros((len(XD_post), 1))

for i in range(len(XD_post)):
    VP = np.exp(-0.5 * (np.mean(((XC_post - XD_post[i,]) / XC_post)**2, axis=1)))
    nc = np.argmax(VP)
    fhat_post[i,] = int(nc)
    XC_post[nc,] = (XC_post[nc,] + XD_post[i,]) / 2


# Estadísticos por cluster POST integración
print("Estadísticos Post Integración")
for j in range(3):  # 3 clusters
    filas_j = np.where(fhat_post[:,] == j)[0]
    arr_j_post = XD_post[filas_j, :] if len(filas_j) > 0 else np.empty((0, XD_post.shape[1]))
    print(f"\nCluster {['A','B','C'][j]} - {len(filas_j)} individuos")
    display(cluster_stats(arr_j_post, variables))


Estadísticos Post Integración

Cluster A - 4251 individuos


,mean,std,skew,kurtosis
Customer_care_calls,4.041637,1.124239,0.381321,2.729029
Customer_rating,3.217596,1.330345,-0.140663,1.839152
Cost_of_the_Product,210.144672,47.381277,-0.134023,2.034240
Prior_purchases,3.578687,1.558614,1.715740,6.957050
Weight_in_gms,3741.510468,1621.758442,-0.384271,1.634221



Cluster B - 1932 individuos


,mean,std,skew,kurtosis
Customer_care_calls,4.103520,1.199710,0.319050,2.436364
Customer_rating,2.818841,1.443677,0.156226,1.674972
Cost_of_the_Product,212.232402,49.040143,-0.160769,1.971549
Prior_purchases,3.625776,1.520861,1.583689,6.766548
Weight_in_gms,3325.389752,1691.491327,0.070214,1.413062



Cluster C - 1149 individuos


,mean,std,skew,kurtosis
Customer_care_calls,3.972150,1.078419,0.406021,2.786452
Customer_rating,2.381201,1.449809,0.593526,1.925790
Cost_of_the_Product,210.991297,46.053842,-0.232657,2.149884
Prior_purchases,3.416014,1.386579,1.771054,7.787229
Weight_in_gms,3751.665796,1523.527360,-0.328250,1.768216


##**Análisis de Resultados**

Al analizar el conjunto de datos, lo primero que se observa es que en general los clientes realizaron en promedio 4 llamadas al servicio al cliente, con una calificación media cercana a 3 estrellas, y un costo de producto alrededor de 210 unidades monetarias. Las compras anteriores rondaron las 3 a 4 veces, y el peso promedio de los productos fue de aproximadamente 3.600 gramos. Esto nos da una visión inicial bastante equilibrada, aunque con cierta dispersión en algunas variables como el peso y el costo del producto.

En cuanto a las variables categóricas, se notó que la mayor parte de los envíos se realizaron a través de barco (cerca del 68%), seguidos por transporte aéreo y terrestre en proporciones similares (alrededor del 16% cada uno). En los bloques de referencia, predominó el bloque C (50%), seguido por A (25%) y B (25%). La importancia de los productos estuvo concentrada en los niveles bajo (48%) y medio (43%), mientras que solo un 9% se catalogó como alta. Finalmente, el género estuvo muy equilibrado: 50,1% femenino y 49,9% masculino.

Pasando a la clusterización, antes de la integración se identificaron tres grupos principales:
- Cluster A, con 3113 individuos, mostró un promedio de 4,77 llamadas al servicio al cliente, una calificación de casi 3,8, y un peso bajo en comparación a los demás (1256 g).
- Cluster B, con 1905 individuos, tuvo una calificación muy baja (1,0), aunque se destacó por un costo de producto relativamente alto (238) y un peso moderado (1375 g).
- Cluster C, con 481 individuos, fue el de mayor peso promedio (4735 g) y el de mayor costo (250), aunque con calificaciones más intermedias (2,9).

Antes de integrar, los grupos se diferenciaban sobre todo por el peso y costo de los productos, además de la calificación.
Cuando se revisan los resultados después de la integración, se observan cambios importantes. El tamaño de los clusters se redistribuyó:

- Cluster A pasó a tener 4251 individuos, manteniendo un perfil muy parecido al inicial, con promedios estables (por ejemplo, 3,21 en calificación, 210 en costo y 3741 g en peso).
- Cluster B se consolidó con 1932 individuos, con una calificación algo mayor (2,82) y un costo levemente superior (212), aunque el peso bajó a 3325 g.
- Cluster C, ahora con 1149 individuos, mantuvo los valores más bajos en calificación (2,38) y compras previas (3,41), pero con un peso alto de 3752 g.

Pese a la integración, los promedios globales del dataset no cambiaron mucho (por ejemplo, la calificación pasó de 2,977 a 2,981, y el costo de producto de 210,7 a 210,8). Sin embargo, a nivel de clusters sí se vieron ajustes: los grupos se volvieron más balanceados en tamaño y los promedios se estabilizaron, lo que indica que la integración ayudó a distribuir mejor a los individuos sin alterar demasiado la esencia de cada grupo.